In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv('law_data.csv')

#Preprocessing
df = df.drop(columns = ['Unnamed: 0'])
df = df[df['region_first'] != 'PO']

#One-Hot encode race
race_coded = pd.get_dummies(df['race'])
df = pd.concat([df, race_coded],axis=1)

#One-Hot encode gender
gender_coded = pd.get_dummies(df['sex'])
gender_coded.columns = ['female', 'male']
df = pd.concat([df, gender_coded],axis=1)

df = df.drop(columns = ['race', 'sex'])
sense_cols = ['Amerindian', 'Asian', 'Black', 'Hispanic', 'Mexican', 'Other','Puertorican', 'White', 'female', 'male']

In [3]:
from sklearn.model_selection import train_test_split
#In our model, ZFYA is something we are interested in, so mark it as y, note that the ZFYA is standardized.

X = df.loc[:, df.columns !='ZFYA']
y = df['ZFYA']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [4]:
n = len(X_train)
ne = len(X_test)
K = len(sense_cols)  #latent variable knowledge that affects gpa, last and fya, but is not related to race and sex

In [5]:
X_train['LSAT'] = X_train['LSAT'].astype(int)
X_test['LSAT'] = X_test['LSAT'].astype(int)

In [6]:
law_stan_data = {
    'N' : n, #number of observations
    'K' : K, #number of attributes
    'a' : np.array(X_train[sense_cols]), #protected variable race and sex
    'ugpa' : np.array(X_train['UGPA']), 
    'lsat' : np.array(X_train['LSAT']),
    'zfya' : np.array(y_train)
       }

In [7]:
#This part defines a model for STAN package to use, the model basically defines all distribution in paper page 8.
#

model= """
data {
  int<lower = 0> N; // number of observations
  int<lower = 0> K; // number of covariates
  matrix[N, K]   a; // sensitive variables
  real           ugpa[N]; // UGPA
  int            lsat[N]; // LSAT
  real           zfya[N]; // ZFYA
  
}

transformed data {
  
 vector[K] zero_K;
 vector[K] one_K;
 
 zero_K = rep_vector(0,K);
 one_K = rep_vector(1,K);

}

parameters {

  vector[N] u;

  real ugpa0;
  real eta_u_ugpa;
  real lsat0;
  real eta_u_lsat;
  real eta_u_zfya;
  
  vector[K] eta_a_ugpa;
  vector[K] eta_a_lsat;
  vector[K] eta_a_zfya;
  
  
  real<lower=0> sigma_g_Sq;
}

transformed parameters  {
 // Population standard deviation (a positive real number)
 real<lower=0> sigma_g;
 // Standard deviation (derived from variance)
 sigma_g = sqrt(sigma_g_Sq);
}

model {
  
  // don't have data about this
  u ~ normal(0, 1);
  
  ugpa0 ~ normal(0, 1);
  eta_u_ugpa ~ normal(0, 1);
  lsat0 ~ normal(0, 1);
  eta_u_lsat ~ normal(0, 1);
  eta_u_zfya ~ normal(0, 1);

  eta_a_ugpa ~ normal(zero_K, one_K);
  eta_a_lsat ~ normal(zero_K, one_K);
  eta_a_zfya ~ normal(zero_K, one_K);

  sigma_g_Sq ~ inv_gamma(1, 1);

  // have data about these
  ugpa ~ normal(ugpa0 + eta_u_ugpa * u + a * eta_a_ugpa, sigma_g);
  lsat ~ poisson(exp(lsat0 + eta_u_lsat * u + a * eta_a_lsat));
  zfya ~ normal(eta_u_zfya * u + a * eta_a_zfya, 1);

}
"""

u -> K
a -> Include all sensitive variables sex and race using real data

In [8]:
import stan
import nest_asyncio
nest_asyncio.apply()
#Using STAN package with the model we defined above, this would gives us the posterior distribution knowledge
#We then use the u we learned to generate samples
posterior = stan.build(model, data=law_stan_data, random_seed=1)
fit = posterior.sample(num_chains= 1, num_samples= 50)

Building...



Building: found in cache, done.Messages from stanc:
Sampling:   0%
Sampling:   0% (1/1050)
Sampling:  10% (100/1050)
Sampling:  19% (200/1050)
Sampling:  29% (300/1050)
Sampling:  38% (400/1050)
Sampling:  48% (500/1050)
Sampling:  57% (600/1050)
Sampling:  67% (700/1050)
Sampling:  76% (800/1050)
Sampling:  86% (900/1050)
Sampling:  95% (1000/1050)
Sampling:  95% (1001/1050)
Sampling: 100% (1050/1050)
Sampling: 100% (1050/1050), done.
Messages received during sampling:
  Gradient evaluation took 0.008761 seconds
  1000 transitions using 10 leapfrog steps per transition would take 87.61 seconds.
  Adjust your expectations accordingly!


In [9]:
U_train = fit['u'].mean(axis = 1)
pd.DataFrame(U_train).to_csv('U_train.csv')

#we trying to get an average of those samples, those are learned using u in trainnig and that would be parameters for test set.

ugpa0 = fit['ugpa0'].mean()
eta_u_ugpa = fit['eta_u_ugpa'].mean()
eta_a_ugpa = fit['eta_a_ugpa'].mean(axis = 1)

lsat0 = fit['lsat0'].mean()
eta_u_lsat = fit['eta_u_lsat'].mean()
eta_a_lsat = fit['eta_a_lsat'].mean(axis = 1)

sigma_g = fit['sigma_g'].mean()

In [10]:
#Here we have similar setting for test, in this part, we have datas learned during training. 

model_u = """
data {
  int<lower = 0> N; // number of observations
  int<lower = 0> K; // number of covariates
  
  matrix[N, K]   a; // sensitive variables
  real           ugpa[N]; // UGPA
  int            lsat[N]; // LSAT
  //real           zfya[N]; // ZFYA
  //int<lower = 0> pass[N]; // PASS
  real           ugpa0;
  real           eta_u_ugpa;
  vector[K]      eta_a_ugpa;
  real           lsat0;
  real           eta_u_lsat;
  vector[K]      eta_a_lsat;
  //real           eta_u_zfya;
  //vector[K]      eta_a_zfya;
  //real           pass0;
  //real           eta_u_pass;
  //vector[K]      eta_a_pass;
  real           sigma_g;
  
 
}


parameters {

  vector[N] u;

}


model {
  
  u ~ normal(0, 1);

  // have data about these
  ugpa ~ normal(ugpa0 + eta_u_ugpa * u + a * eta_a_ugpa, sigma_g); 
  lsat ~ poisson(exp(lsat0 + eta_u_lsat * u + a * eta_a_lsat)); 
  //zfya ~ normal(eta_u_zfya * u + a * eta_a_zfya,1);
  //pass ~ bernoulli_logit(pass0 + eta_u_pass * u + a * eta_a_pass);
  
}
"""

In [11]:
#Those are data we learned from training

law_stan_test_data = {
    'N' : ne,#number of test data
    'K' : K,#number of protected attributesv
    'a' : np.array(X_test[sense_cols]), #protected variable race and sex
    'ugpa' : np.array(X_test['UGPA']), 
    'lsat' : np.array(X_test['LSAT']),
    #those are learned from training
    'ugpa0' : ugpa0,
    'eta_u_ugpa' : eta_u_ugpa,
    'eta_a_ugpa' : eta_a_ugpa,
    'lsat0' : lsat0, 
    'eta_u_lsat' : eta_u_lsat,
    'eta_a_lsat' : eta_a_lsat,
    'sigma_g' : sigma_g    
}

In [12]:
posterior_test = stan.build(model_u, data=law_stan_test_data, random_seed=1)
fit_test = posterior_test.sample(num_chains= 1, num_samples=50)

Building...



Building: found in cache, done.Sampling:   0%
Sampling:   0% (1/1050)
Sampling:  10% (100/1050)
Sampling:  19% (200/1050)
Sampling:  29% (300/1050)
Sampling:  38% (400/1050)
Sampling:  48% (500/1050)
Sampling:  57% (600/1050)
Sampling:  67% (700/1050)
Sampling:  76% (800/1050)
Sampling:  86% (900/1050)
Sampling:  95% (1000/1050)
Sampling:  95% (1001/1050)
Sampling: 100% (1050/1050)
Sampling: 100% (1050/1050), done.
Messages received during sampling:
  Gradient evaluation took 0.001374 seconds
  1000 transitions using 10 leapfrog steps per transition would take 13.74 seconds.
  Adjust your expectations accordingly!


In [14]:
#This would be the knowledge we learned from testing by taking average of all samples

U_test = fit_test['u'].mean(axis = 1)

# Fair Model

In [15]:
# 2. Fair + non-deterministic model (deterministic)
from sklearn.linear_model import LinearRegression

X_fair = U_train.reshape(-1,1)
X_fair_test = U_test.reshape(-1, 1)

lr_fair = LinearRegression()
lr_fair.fit(X_fair, y_train)

LinearRegression()

# Unfair models


In [18]:
#A. Full model
full_list = ['LSAT', 'UGPA', 'Amerindian', 'Asian', 'Black', 'Hispanic', 'Mexican','Other', 'Puertorican', 'White', 'female', 'male']
lr_full = LinearRegression()
lr_full.fit(X_train[full_list], y_train)

#B. Unaware model
unaware = ['LSAT', 'UGPA']
lr_unaware = LinearRegression()
lr_unaware.fit(X_train[unaware], y_train)


LinearRegression()

In [21]:
#Compile to one dataset
X_train['ZFYA'] = y_train
X_train['Knowledge'] = U_train

X_train['Init_class'] = np.sign(X_train['ZFYA'])

#fair model
X_train['Fair_pred'] = lr_fair.predict(X_fair)
X_train['Fair_pred_class'] = np.sign(X_train['Fair_pred'])

#full model
X_train['full_pred'] = lr_full.predict(X_train[full_list])
X_train['full_pred_class'] = np.sign(X_train['full_pred'])

#unaware model
X_train['unaware_pred'] = lr_unaware.predict(X_train[unaware])
X_train['unaware_pred_class'] = np.sign(X_train['unaware_pred'])

X_train.to_csv('Full_Training_df.csv')

In [22]:
X_test['ZFYA'] = y_test
X_test['Knowledge'] = U_test

X_test['Init_class'] = np.sign(X_test['ZFYA'])

X_test['Fair_pred'] = lr_fair.predict(X_fair_test)
X_test['Fair_pred_class'] = np.sign(X_test['Fair_pred'])

X_test['full_pred'] = lr_full.predict(X_test[full_list])
X_test['full_pred_class'] = np.sign(X_test['full_pred'])

X_test['unaware_pred'] = lr_unaware.predict(X_test[unaware])
X_test['unaware_pred_class'] = np.sign(X_test['unaware_pred'])


In [48]:
X_test.to_csv('Full_Test_df.csv')

In [31]:
X_test.columns

Index(['LSAT', 'UGPA', 'region_first', 'sander_index', 'first_pf',
       'Amerindian', 'Asian', 'Black', 'Hispanic', 'Mexican', 'Other',
       'Puertorican', 'White', 'female', 'male', 'ZFYA', 'Knowledge',
       'Init_class', 'Fair_pred', 'Fair_pred_class', 'full_pred',
       'full_pred_class', 'unaware_pred', 'unaware_pred_class'],
      dtype='object')

# Counterfactual Data

In [38]:
#Gender counterfactual
male_counter = X_test['female']
female_counter = X_test['male']

test_counter = X_test.copy()
test_counter['male'] = male_counter
test_counter['female'] = female_counter

In [44]:
#get everything and male
test_counter = test_counter.loc[:, 'LSAT':'male']

In [45]:
law_stan_counter_data = {
    'N' : ne,
    'K' : K,
    'a' : np.array(test_counter[sense_cols]), #protected variable race and sex
    'ugpa' : np.array(test_counter['UGPA']), 
    'lsat' : np.array(test_counter['LSAT']),
    'ugpa0' : ugpa0,
    'eta_u_ugpa' : eta_u_ugpa,
    'eta_a_ugpa' : eta_a_ugpa,
    'lsat0' : lsat0, 
    'eta_u_lsat' : eta_u_lsat,
    'eta_a_lsat' : eta_a_lsat,
    'sigma_g' : sigma_g    
}

counter_post = stan.build(model_u, data=law_stan_counter_data, random_seed=1)
counter_fit = counter_post.sample(num_chains= 1, num_samples=50)
counter_U = counter_fit['u'].mean(axis = 1)
counter_fair_K = counter_U.reshape(-1, 1)

counter_fair = LinearRegression()
counter_fair.fit(counter_fair_K, y_test)

Building...



Building: found in cache, done.Sampling:   0%
Sampling:   0% (1/1050)
Sampling:  10% (100/1050)
Sampling:  19% (200/1050)
Sampling:  29% (300/1050)
Sampling:  38% (400/1050)
Sampling:  48% (500/1050)
Sampling:  57% (600/1050)
Sampling:  67% (700/1050)
Sampling:  76% (800/1050)
Sampling:  86% (900/1050)
Sampling:  95% (1001/1050)
Sampling: 100% (1050/1050)
Sampling: 100% (1050/1050), done.
Messages received during sampling:
  Gradient evaluation took 0.00178 seconds
  1000 transitions using 10 leapfrog steps per transition would take 17.8 seconds.
  Adjust your expectations accordingly!


LinearRegression()

In [ ]:
test_counter['ZFYA'] = y_test
test_counter['Knowledge'] = counter_fair_K
test_counter['Init_class'] = np.sign(X_test['ZFYA'])


#knowledge is not affected by gender, direct prediction
test_counter['Fair_pred'] = counter_fair.predict(counter_fair_K)
test_counter['Fair_pred_class'] = np.sign(test_counter['Fair_pred'])

In [99]:
#Gender counterfactual
train_counter = X_train.copy()

male_counter = X_train['female']
female_counter = X_train['male']

train_counter['male'] = male_counter
train_counter['female'] = female_counter

#Counterfactual LSAT and UGPA - by causal model, affected by gender, need to learn first
counter_LSAT = LinearRegression(fit_intercept = True)
counter_LSAT.fit(train_counter[sense_cols], train_counter['LSAT'])

counter_UGPA = LinearRegression(fit_intercept = True)
counter_UGPA.fit(train_counter[sense_cols], train_counter['UGPA'])

#counter_ZFYA = LinearRegression(fit_intercept = True)
#counter_ZFYA.fit(train_counter[sense_cols], train_counter['ZFYA'])

LinearRegression()

In [100]:
#get counterfactual LSAT and UGPA
test_counter['LSAT'] = counter_LSAT.predict(test_counter[sense_cols])
test_counter['UGPA'] = counter_UGPA.predict(test_counter[sense_cols])
#y_test_counter = counter_ZFYA.predict(test_counter[sense_cols])

In [101]:
#Counterfactual full
test_counter['full_pred'] = lr_full.predict(test_counter[full_list])
test_counter['full_pred_class'] = np.sign(test_counter['full_pred'])

In [ ]:
#Counterfactual unaware
test_counter['unaware_pred'] = lr_unaware.predict(test_counter[unaware])
test_counter['unaware_pred_class'] = np.sign(test_counter['unaware_pred'])

In [102]:
test_counter.to_csv('Counter_Test_df.csv')

# Metrics

In [106]:
df1 = pd.read_csv('Full_test_df.csv')
df2 = pd.read_csv('Counter_test_df.csv')
mapping = {-1: 'unsuccessful', 1: 'successful'}

result = pd.DataFrame({
    'Gender': df1['female'].map({0: 'male', 1:'female'}),
    'Init_class': df1['Init_class'].map(mapping),
    
    'Fair_pred_class': df1['Fair_pred_class'].map(mapping),
    'Counter_fair_pred_class': df2['Fair_pred_class'].map(mapping),
    
    'full_pred_class':df1['full_pred_class'].map(mapping),
    'Counter_full_pred_class': df2['full_pred_class'].map(mapping),
    
    'unaware_pred_class':df1['unaware_pred_class'].map(mapping),
    'Counter_unaware_pred_class': df2['unaware_pred_class'].map(mapping)
})

# Female: Predicted probability to succeed 

In [107]:
female = result[result['Gender'] == 'female']
male = result[result['Gender'] == 'male']   
n_female = len(female)
n_male = len(male)

In [108]:
pd.DataFrame({
    'Proability': [len(female[female['full_pred_class'] == 'successful'])/n_female, 
                   len(female[female['unaware_pred_class'] == 'successful'])/n_female,
                   len(female[female['Fair_pred_class'] == 'successful'])/n_female
                  ],
    'Counterfactual Probabilty': [len(female[female['Counter_full_pred_class'] == 'successful'])/n_female,
                                 len(female[female['Counter_unaware_pred_class'] == 'successful'])/n_female,
                                 len(female[female['Counter_fair_pred_class'] == 'successful'])/n_female]
}, index = ['Full Model', 'Unaware Model', 'Fair Model(learned)'])

,Proability,Counterfactual Probabilty
Full Model,0.718220,0.806674
Unaware Model,0.661547,0.863347
Fair Model(learned),0.736758,0.934322


# Male: Predicted probability to succeed 

In [109]:
pd.DataFrame({
    'Proability': [len(male[male['full_pred_class'] == 'successful'])/n_male, 
                   len(male[male['unaware_pred_class'] == 'successful'])/n_male,
                   len(male[male['Fair_pred_class'] == 'successful'])/n_male
                  ],
    'Counterfactual Probabilty': [len(male[male['Counter_full_pred_class'] == 'successful'])/n_male,
                                 len(male[male['Counter_unaware_pred_class'] == 'successful'])/n_male,
                                 len(male[male['Counter_fair_pred_class'] == 'successful'])/n_male]
}, index = ['Full Model', 'Unaware Model', 'Fair Model(learned)'])

,Proability,Counterfactual Probabilty
Full Model,0.759109,0.862753
Unaware Model,0.650607,0.913360
Fair Model(learned),0.729960,0.794332
